In [9]:
import os
import re
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from sentence_transformers import SentenceTransformer
import chromadb
from concurrent.futures import ThreadPoolExecutor
import torch
import clip
import numpy as np
from tqdm import tqdm
import shutil

In [2]:
#!pip install git+https://github.com/openai/CLIP.git
#!pip install pillow requests pandas sentence-transformers chromadb
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [14]:
df = pd.read_csv("amazon_dataset.csv")
print(df.shape)

(10002, 28)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                10002 non-null  object 
 1   Product Name           10002 non-null  object 
 2   Brand Name             0 non-null      float64
 3   Asin                   0 non-null      float64
 4   Category               9172 non-null   object 
 5   Upc Ean Code           34 non-null     object 
 6   List Price             0 non-null      float64
 7   Selling Price          9895 non-null   object 
 8   Quantity               0 non-null      float64
 9   Model Number           8230 non-null   object 
 10  About Product          9729 non-null   object 
 11  Product Specification  8370 non-null   object 
 12  Technical Details      9212 non-null   object 
 13  Shipping Weight        8864 non-null   object 
 14  Product Dimensions     479 non-null    object 
 15  Im

In [16]:
# Keeping only that columns that are optimal to provide information on products
columns_to_keep = [
    'Uniq Id',
    'Product Name',
    'Category',
    'Selling Price',
    'About Product',
    'Product Specification',
    'Technical Details',
    'Shipping Weight',
    'Image',
    'Product Url',
    'Is Amazon Seller'
]

df_clean = df[columns_to_keep].copy()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Uniq Id                10002 non-null  object
 1   Product Name           10002 non-null  object
 2   Category               9172 non-null   object
 3   Selling Price          9895 non-null   object
 4   About Product          9729 non-null   object
 5   Product Specification  8370 non-null   object
 6   Technical Details      9212 non-null   object
 7   Shipping Weight        8864 non-null   object
 8   Image                  10002 non-null  object
 9   Product Url            10002 non-null  object
 10  Is Amazon Seller       10002 non-null  object
dtypes: object(11)
memory usage: 859.7+ KB


In [17]:
df_clean.head()

,Uniq Id,Product Name,Category,Selling Price,About Product,Product Specification,Technical Details,Shipping Weight,Image,Product Url,Is Amazon Seller
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",Sports & Outdoors | Outdoor Recreation | Skate...,$237.68,Make sure this fits by entering your model num...,Shipping Weight: 10.7 pounds (View shipping ra...,NaN,10.7 pounds,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/DB-Longboards-CoreFlex-...,Y
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",Toys & Games | Learning & Education | Science ...,$99.95,Make sure this fits by entering your model num...,Product Dimensions: 14.7 x 11.1 x 10.2...,The snap circuits mini kits classpack provides...,4 pounds,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Electronic-Circuits-Cla...,Y
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,Toys & Games | Arts & Crafts | Craft Kits,$34.99,Make sure this fits by entering your model num...,ProductDimensions:10.3x3.4x0.8inches|ItemWeigh...,show up to 2 reviews by default No longer are ...,12.8 ounces,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/3Doodler-Plastic-Innova...,Y
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,Toys & Games | Hobbies | Models & Model Kits |...,$28.91,Make 8 different Planes at one time. | Experim...,ProductDimensions:3.5x6.2x13inches|ItemWeight:...,Go to your orders and start the return Select ...,13.4 ounces,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Guillow-Airplane-Design...,Y
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,Toys & Games | Puzzles | Jigsaw Puzzles,$17.49,Make sure this fits by entering your model num...,ProductDimensions:1.9x8x10inches|ItemWeight:13...,show up to 2 reviews by default 100% Officiall...,13.4 ounces,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Woodstock-Collage-500-p...,Y


## Text Data Pre-processing

### Clean selling price column

In [19]:
problem_rows = df_clean[df_clean['Selling Price']
    .astype(str)
    .str.replace(r'[^0-9.]', '', regex=True)
    .str.contains(r'\d+\.\d+\d+\.\d+')]  # two decimal points

In [20]:
print(problem_rows)

                               Uniq Id  \
25    6d38858169064c8b7069a19c90dd0ae4   
39    c3296d8ad42c3998c205f0dbca3998f3   
150   d97a81d04b9463ebced3488341e8df6a   
166   44af50aecebcfbca03654d2ab92aac6b   
173   5ec9d590a90fce754b08cc415627ef07   
...                                ...   
9941  88734f8decae022f6584d4d85f1952d2   
9952  f5e1ea4bbf6bf1903f3dfcb0c0497cb7   
9960  f53fc55470d36bc5cb675eb665631b64   
9977  68f93eb7852263685cb7259ff453db05   
9989  80a95e540e4b2bcb59ecb9d8040a9a48   

                                           Product Name  \
25    Huffy Kids Bikes 16 & 20 inch with Streamers a...   
39    Poopsie Unicorn Crush with Glitter and Slime S...   
150             Medieval Princess Fuschia Child Costume   
166   Transformers Bumblebee Movie Muscle Bumblebee ...   
173   Snow White and The Huntsman Deluxe Queen Raven...   
...                                                 ...   
9941  Noch 61230 Rock Mold "Zugspitze" G,0,H0,TT,N,Z...   
9952  UST Learn & Live 

Since we see that the prices sometimes have a range, duplicated price patterns, and unusual spaces and dots, we'll set up a cleaning function to take care of this. The function will output a minimum and maximum price to ensure every row has min and max selling price, so later code doesn’t break depending on price type.

Prompt used in ChatGPT for code in this section: "How can I write a Python function to clean a price column that may contain ranges, multiple dollar signs, or weird spacing? I want to extract all numeric values (floats) and return min and max prices."

In [21]:
import re
import pandas as pd

def clean_selling_price(price_str):
    if not isinstance(price_str, str) or not price_str.strip():
        return None, None  # No price

    # Removig repeated currency symbols and normalize spaces
    price_str = re.sub(r'\$', '', price_str)
    price_str = re.sub(r'\s+', ' ', price_str.strip())

    # Extractig all numeric values
    numbers = re.findall(r'\d+\.\d+', price_str)
    if not numbers:
        return None, None

    # Converting to floats
    numbers = [float(num) for num in numbers]

    # If it's a range, return min and max
    if len(numbers) >= 2:
        return min(numbers), max(numbers)
    else:
        return numbers[0], numbers[0]

df_clean[['Minimum Selling Price', 'Maximum Selling Price']] = df_clean['Selling Price'].apply(lambda x: pd.Series(clean_selling_price(x)))
print(df_clean)

                                Uniq Id  \
0      4c69b61db1fc16e7013b43fc926e502d   
1      66d49bbed043f5be260fa9f7fbff5957   
2      2c55cae269aebf53838484b0d7dd931a   
3      18018b6bc416dab347b1b7db79994afa   
4      e04b990e95bf73bbe6a3fa09785d7cd0   
...                                 ...   
9997   1a22f23576bfdfe5ed6c887dc117aab6   
9998   e11514dcf1f087887cd5ea0bd646d1fc   
9999   c00301a38560da2abc89c1f86ce4b267   
10000  c2928dbf9796ceba44863a2736afb405   
10001  2bb94aefc3467ed83860e0e2712d5f10   

                                            Product Name  \
0      DB Longboards CoreFlex Crossbow 41" Bamboo Fib...   
1      Electronic Snap Circuits Mini Kits Classpack, ...   
2      3Doodler Create Flexy 3D Printing Filament Ref...   
3      Guillow Airplane Design Studio with Travel Cas...   
4                       Woodstock- Collage 500 pc Puzzle   
...                                                  ...   
9997   Remedia Publications REM536B Money Activity Bo...   
999

### Parsing shipping weight

Prompt used from ChatGPT for code below: "I have a shipping weight column with values like '10.7 pounds', '12.8 ounces', and sometimes bad values like '. pounds'. How can I detect rows where regex parsing fails (bad matches)?"

In [22]:
df_clean['Shipping Weight'].dropna().unique()[:50]

array(['10.7 pounds', '4 pounds', '12.8 ounces', '13.4 ounces',
       '1.4 pounds', '9.8 ounces', '12.2 ounces', '1.5 pounds',
       '3.2 ounces', '2.1 pounds', '5.6 ounces', '0.16 ounces',
       '2.9 pounds', '39 pounds', '15.2 ounces', '4.2 ounces', '2 pounds',
       '1.6 ounces', '5.3 pounds', '11.2 ounces', '1.23 pounds',
       '1.37 pounds', '13.6 ounces', '1.61 pounds', '0.8 ounces',
       '3.36 ounces', '4.23 pounds', '1.45 pounds', '1.12 ounces',
       '4.8 ounces', '1.3 pounds', '4.6 ounces', '1.64 pounds',
       '6.4 pounds', '0.32 ounces', '8.5 ounces', '8 ounces', '5 ounces',
       '2.08 ounces', '3.52 ounces', '1.92 ounces', '12.6 ounces',
       '6.2 ounces', '12 ounces', '1.54 pounds', '3.7 pounds',
       '8.8 pounds', '0.48 ounces', '10.4 ounces', '14.4 ounces'],
      dtype=object)

In [23]:
bad_rows = []
for val in df_clean['Shipping Weight'].dropna():
    m = re.search(r'([\d.]+)\s*([a-zA-Z]+)', str(val))
    if m:
        try:
            float(m.group(1))
        except ValueError:
            bad_rows.append(val)

bad_rows = list(set(bad_rows))  # unique
print("Bad matches:", bad_rows)


Bad matches: ['. pounds']


Prompt used from ChatGPT for code below: "Can you help me write a function to parse product weights from text like '12.8 ounces' or '1.5 pounds'? I need to extract the numeric value and the unit separately."

In [24]:
def parse_weight(text):
    if not isinstance(text, str):
        return None, None

    # Matching number with optional decimal, must have at least one digit
    m = re.search(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+)', text)
    if m:
        try:
            return float(m.group(1)), m.group(2).lower()
        except ValueError:
            return None, None
    return None, None

df_clean[['Shipping Weight Value', 'Shipping Weight Unit']] = df_clean['Shipping Weight'].apply(
    lambda x: pd.Series(parse_weight(x))
)

In [25]:
def to_pounds(value, unit):
    if pd.isna(value) or pd.isna(unit):
        return None
    if unit == "pounds":
        return value
    elif unit == "ounces":
        return value / 16
    return None

df_clean["Shipping Weight (lb)"] = df_clean.apply(lambda row: to_pounds(row["Shipping Weight Value"], row["Shipping Weight Unit"]), axis=1)

In [26]:
df_clean['Shipping Weight Unit'].dropna().unique()[:50]

array(['pounds', 'ounces'], dtype=object)

In [27]:
print(df_clean)

                                Uniq Id  \
0      4c69b61db1fc16e7013b43fc926e502d   
1      66d49bbed043f5be260fa9f7fbff5957   
2      2c55cae269aebf53838484b0d7dd931a   
3      18018b6bc416dab347b1b7db79994afa   
4      e04b990e95bf73bbe6a3fa09785d7cd0   
...                                 ...   
9997   1a22f23576bfdfe5ed6c887dc117aab6   
9998   e11514dcf1f087887cd5ea0bd646d1fc   
9999   c00301a38560da2abc89c1f86ce4b267   
10000  c2928dbf9796ceba44863a2736afb405   
10001  2bb94aefc3467ed83860e0e2712d5f10   

                                            Product Name  \
0      DB Longboards CoreFlex Crossbow 41" Bamboo Fib...   
1      Electronic Snap Circuits Mini Kits Classpack, ...   
2      3Doodler Create Flexy 3D Printing Filament Ref...   
3      Guillow Airplane Design Studio with Travel Cas...   
4                       Woodstock- Collage 500 pc Puzzle   
...                                                  ...   
9997   Remedia Publications REM536B Money Activity Bo...   
999

### Creating a top category column

In [28]:
df_clean['Top Category'] = df_clean['Category'].astype(str).str.split('|').str[0].str.strip()

In [29]:
print(df_clean)

                                Uniq Id  \
0      4c69b61db1fc16e7013b43fc926e502d   
1      66d49bbed043f5be260fa9f7fbff5957   
2      2c55cae269aebf53838484b0d7dd931a   
3      18018b6bc416dab347b1b7db79994afa   
4      e04b990e95bf73bbe6a3fa09785d7cd0   
...                                 ...   
9997   1a22f23576bfdfe5ed6c887dc117aab6   
9998   e11514dcf1f087887cd5ea0bd646d1fc   
9999   c00301a38560da2abc89c1f86ce4b267   
10000  c2928dbf9796ceba44863a2736afb405   
10001  2bb94aefc3467ed83860e0e2712d5f10   

                                            Product Name  \
0      DB Longboards CoreFlex Crossbow 41" Bamboo Fib...   
1      Electronic Snap Circuits Mini Kits Classpack, ...   
2      3Doodler Create Flexy 3D Printing Filament Ref...   
3      Guillow Airplane Design Studio with Travel Cas...   
4                       Woodstock- Collage 500 pc Puzzle   
...                                                  ...   
9997   Remedia Publications REM536B Money Activity Bo...   
999

### Creating a column that flags if a product image exists or not

Prompt used from ChatGPT for code below: "How can I write a function in Python to check if an image URL is valid (exists)? Should I use requests or another package to check the status code efficiently?" Can you also show me how to run these checks in parallel for a DataFrame column of URLs?"

In [30]:
def image_exists(url):
    try:
        r = requests.head(url, timeout=5)
        return r.status_code == 200
    except:
        return False

# Parallel checking with 20 threads
def check_images_parallel(urls, max_workers=20):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(image_exists, urls))
    return results

# Applying in parallel and assigning to DataFrame
df_clean['Image Exists'] = [
    'Y' if exists else 'N'
    for exists in check_images_parallel(df_clean['Image'])
]

In [31]:
df_clean['Image Exists'].value_counts()

,count
Image Exists,
Y,9970
N,32


### Cleaning About Products, Product Description and Technical Details

In [32]:
df_clean['About Product'].unique()[:10]

array(["Make sure this fits by entering your model number. | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern that’s second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds. | COREFLEX TECH: CoreFlex construction is water resistant, impact resistant, scratch resistant and has a flex like you won’t believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength. | INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the Northwest | BEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like family | BEYOND COMPARE: Try our skatebo

In [33]:
df_clean['Product Specification'].unique()[:10]

array(['Shipping Weight: 10.7 pounds (View shipping rates and policies)|ASIN: B07KMVJJK7|    #474    in\xa0Longboards Skateboard',
       'Product Dimensions:         14.7 x 11.1 x 10.2 inches ; 4.06 pounds    |Shipping Weight: 4 pounds (View shipping rates and policies)|Domestic Shipping: Item can be shipped within U.S.|International Shipping: This item can be shipped to select countries outside of the U.S.  Learn More|ASIN: B008AK6DAS|Item model number: 55324|    #3032    in\xa0Science Kits & Toys',
       'ProductDimensions:10.3x3.4x0.8inches|ItemWeight:12.8ounces|ShippingWeight:12.8ounces(Viewshippingratesandpolicies)|ASIN:B07D36747F|Manufacturerrecommendedage:14yearsandup',
       'ProductDimensions:3.5x6.2x13inches|ItemWeight:13.4ounces|ShippingWeight:13.4ounces(Viewshippingratesandpolicies)|DomesticShipping:ItemcanbeshippedwithinU.S.|InternationalShipping:ThisitemcanbeshippedtoselectcountriesoutsideoftheU.S.LearnMore|ASIN:B076Y2SNHM|Itemmodelnumber:142|Manufacturerrecommendedage

In [34]:
df_clean['Technical Details'].unique()[:10]

array([nan,
       'The snap circuits mini kits classpack provides basic electric circuitry information for students in grades 2-6. This classpack includes 5 snap-together building kits. Components snap together to create working models of everyday electronic devices. Kits included are an FM radio, a motion detector, a music box, space battle sound effects, and a flying saucer. Each mini kit comes with individual components, and an activity guide which includes instructions and additional project ideas. Each primary-colored component represents one function in a circuit. Activity kits are used by teachers and students in classroom and homeschool settings for educational and research applications in science, math, and for a variety of additional disciplines. Science education products and manipulatives incorporate applied math and science principles into classroom or homeschool projects. Teachers in pre-K, elementary, and secondary classrooms use science education kits, manipualtives, a

Prompt used from ChatGPT for code below: "I need to clean text fields like 'About Product', 'Technical Details' and 'Product Specification'. Can you help me write a regex patterns to remove boilerplate phrases (i will create a list of these), fix stuck-together words and ensure spacing around units (like '12.8ounces' → '12.8 ounces'), remove emojis, break camel case words, create a new line bulletpoint when you see •, newline, or semicolon, capitalize first word of each bullet point. I will create a function of these, including some extra clean up"

In [35]:
BOILERPLATE_PATTERNS = [
    r"make sure this fits by entering your model number\.?",
    r"view shipping rates and policies",
    r"product description",
    r"go to your orders and start the return select the ship method ship it!?",
    r"show up to \d+ reviews by default",
    r"officially licensed.*?(?=•|$)",
    r"domestic shipping.*?(?=•|$)",
    r"international shipping.*?(?=•|$)"
]

# Words for unit spacing
UNITS = ["pounds", "pound", "lbs", "lb", "ounces", "ounce", "oz",
         "inches", "inch", "cm", "mm", "kg", "g"]

def fix_stuck_words(t):
    # Add space after colon if missing
    t = re.sub(r":(?=\S)", ": ", t)

    # Add space after period if missing
    t = re.sub(r"\.(?=[A-Z0-9])", ". ", t)

    # Break camelCase / PascalCase / mixedCase words
    t = re.sub(r"([a-z])([A-Z])", r"\1 \2", t)  # lower→Upper
    t = re.sub(r"([A-Z][a-z])([A-Z])", r"\1 \2", t)  # UpperLower→Upper
    t = re.sub(r"(?<=[a-zA-Z])(?=\d)", " ", t)  # letter→digit
    t = re.sub(r"(?<=\d)(?=[a-zA-Z])", " ", t)  # digit→letter

    # Break known long jammed phrases
    jammed_phrases = {
        "Manufacturerrecommendedage": "Manufacturer recommended age",
        "Itemcanbeshippedwithin": "Item can be shipped within",
        "Thisitemcanbeshippedtoselectcountries": "This item can be shipped to select countries",
        "Viewshippingratesandpolicies": "View shipping rates and policies",
        "MadeinUSA": "Made in USA",
        "Thisitemmustbeshippedseparatelyfromotheritemsinyourorder" : "This item must be shipped separately from other items in your order",
        "Additionalshippingchargeswillnotapply": "Additional shipping charges will not apply",
        "Itemmodelnumber" : "Item model number",
        "yearsandup" : "years and up"

    }
    for bad, good in jammed_phrases.items():
        t = t.replace(bad, good)

    # Ensure spacing before known keywords
    keywords = [
        "ASIN", "Manufacturer", "Itemmodelnumber", "Item model number",
        "Shipping Weight", "Item Weight", "Product Dimensions",
        "Domestic Shipping", "International Shipping", "Manufacturer recommended age",
        "Item can be shipped", "This item can be shipped",
        "Learn More", "Made in USA", "View shipping rates and policies"
    ]
    for kw in keywords:
        t = re.sub(r"(?<!\s)(" + re.escape(kw) + r")", r" \1", t)

    # Space between numbers and measurement units
    unit_pattern = r"(\d+(?:\.\d+)?)(?=\s?(" + "|".join(UNITS) + r"))"
    t = re.sub(unit_pattern, r"\1 ", t)

    return t

def clean_text_block(text):
    if not isinstance(text, str) or not text.strip() or text.strip().lower() == "information unavailable":
        return ""

    t = text

    # Remove boilerplate
    for pat in BOILERPLATE_PATTERNS:
        t = re.sub(pat, "", t, flags=re.I)

    # Remove all emoji and non-standard symbols (keep only letters, numbers, spaces, basic punctuation)
    t = re.sub(r"[^a-zA-Z0-9\s\.,;:!?%&\-/()'\"]+", "", t)

    # Fix stuck words and units
    t = fix_stuck_words(t)

    # Remove empty parentheses left behind
    t = re.sub(r"\(\s*\)", "", t)

    # Replace | with newline for separation
    t = t.replace("|", "\n")

    # Normalize spaces
    t = re.sub(r"\s+", " ", t)

    # Split into bullet points (•, newline, semicolon)
    parts = re.split(r"(?:•|\n|;)", t)
    parts = [p.strip(" .,-") for p in parts if p.strip()]

    # Capitalize first letter of each bullet
    cleaned_parts = [p[0].upper() + p[1:] if p else "" for p in parts]

    return "\n".join(f"• {p}" for p in cleaned_parts if p)

In [36]:
# Applying cleaning functions
df_clean['About Product Clean'] = df_clean['About Product'].apply(clean_text_block)
df_clean['Product Specification Clean'] = df_clean['Product Specification'].apply(clean_text_block)
df_clean['Technical Details Clean'] = df_clean['Technical Details'].apply(clean_text_block)

In [37]:
df_clean['About Product Clean'].unique()[:10]

array(["• RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern thats second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds. COREFLEX TECH: Core Flex construction is water resistant, impact resistant, scratch resistant and has a flex like you wont believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength. INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the Northwest BEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like family BEYOND COMPARE: Try our skateboards & accessories if you've tried similar products by Secto

In [38]:
df_clean['Product Specification Clean'].unique()[:10]

array(['• Shipping Weight: 10. 7 pounds ASIN: B 07 KMVJJK 7 474 in Longboards Skateboard',
       '• Product Dimensions: 14. 7 x 11. 1 x 10. 2 inches\n• 4. 06 pounds Shipping Weight: 4 pounds',
       '• Product Dimensions: 10. 3 x 3. 4 x 0. 8 inches Item Weight: 12. 8 ounces Shipping Weight: 12. 8 ounces( View shipping rates and policies) ASIN: B 07 D 36747 F Manufacturer recommended age: 14 years and up',
       '• Product Dimensions: 3. 5 x 6. 2 x 13 inches Item Weight: 13. 4 ounces Shipping Weight: 13. 4 ounces( View shipping rates and policies) Domestic Shipping: Item can be shipped within U. S. International Shipping: This item can be shipped to select countriesoutsideofthe U. S. Learn More ASIN: B 076 Y 2 SNHM Item model number: 142 Manufacturer recommended age: 8 years and up',
       '• Product Dimensions: 1. 9 x 8 x 10 inches Item Weight: 13. 4 ounces Shipping Weight: 13. 4 ounces( View shipping rates and policies) ASIN: B 07 MX 21 WWX Item model number: 62151 Manufacturer re

In [39]:
df_clean['Technical Details Clean'].unique()[:10]

array(['',
       '• The snap circuits mini kits classpack provides basic electric circuitry information for students in grades 2-6. This classpack includes 5 snap-together building kits. Components snap together to create working models of everyday electronic devices. Kits included are an FM radio, a motion detector, a music box, space battle sound effects, and a flying saucer. Each mini kit comes with individual components, and an activity guide which includes instructions and additional project ideas. Each primary-colored component represents one function in a circuit. Activity kits are used by teachers and students in classroom and homeschool settings for educational and research applications in science, math, and for a variety of additional disciplines. Science education products and manipulatives incorporate applied math and science principles into classroom or homeschool projects. Teachers in pre-K, elementary, and secondary classrooms use science education kits, manipualtives, 

In [40]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Uniq Id                      10002 non-null  object 
 1   Product Name                 10002 non-null  object 
 2   Category                     9172 non-null   object 
 3   Selling Price                9895 non-null   object 
 4   About Product                9729 non-null   object 
 5   Product Specification        8370 non-null   object 
 6   Technical Details            9212 non-null   object 
 7   Shipping Weight              8864 non-null   object 
 8   Image                        10002 non-null  object 
 9   Product Url                  10002 non-null  object 
 10  Is Amazon Seller             10002 non-null  object 
 11  Minimum Selling Price        9837 non-null   float64
 12  Maximum Selling Price        9837 non-null   float64
 13  Shipping Weight 

In [41]:
# Filling NaNs
df_clean = df_clean.fillna("Information unavailable")

In [42]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Uniq Id                      10002 non-null  object
 1   Product Name                 10002 non-null  object
 2   Category                     10002 non-null  object
 3   Selling Price                10002 non-null  object
 4   About Product                10002 non-null  object
 5   Product Specification        10002 non-null  object
 6   Technical Details            10002 non-null  object
 7   Shipping Weight              10002 non-null  object
 8   Image                        10002 non-null  object
 9   Product Url                  10002 non-null  object
 10  Is Amazon Seller             10002 non-null  object
 11  Minimum Selling Price        10002 non-null  object
 12  Maximum Selling Price        10002 non-null  object
 13  Shipping Weight Value        10

## Creating Final Text for Embeddings and Building the Metadata for Filtering

In [44]:
# This is the unstructured information of a product that the LLM will read.
# It combines all the relevant text in a way that's human-readable and search-friendly.
def build_context_text(row):
    return f"Title: {row['Product Name']}"

df_clean['context_text'] = df_clean.apply(build_context_text, axis=1)

In [47]:
df_clean['context_text'].unique()

array(['Title: DB Longboards CoreFlex Crossbow 41" Bamboo Fiberglass Longboard Complete',
       'Title: Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5)',
       "Title: 3Doodler Create Flexy 3D Printing Filament Refill Bundle (X5 Pack, Over 1000'. of Extruded Plastics! - Innovate",
       ...,
       'Title: NewPath Learning 10 Piece Science Owls and Owl Pellets Curriculum Mastery Flip Chart Set, Grade 5-9',
       'Title: Disney Princess Do It Yourself Braid Set',
       'Title: Hasegawa Ladders Lucano Step Ladder, Orange'], dtype=object)

In [46]:
# Metadata contains facts about a product that we can filter/sort on in RAG without using embeddings.

def build_metadata(row):
    return {
        "unique_id": row['Uniq Id'],
        "product_name": row['Product Name'],
        "top_category": row['Top Category'],
        "category": row['Category'],
        "selling_price_min": row['Minimum Selling Price'],
        "selling_price_max": row['Maximum Selling Price'],
        "shipping_weight_lb": row['Shipping Weight (lb)'],  # numeric, for numeric filters
        "shipping_weight_value": row['Shipping Weight Value'],  # original parsed value
        "shipping_weight_unit": row['Shipping Weight Unit'],    # original parsed unit
        "is_amazon_seller": bool(row['Is Amazon Seller']),
        "image_exists": bool(row['Image Exists']),
        "image_url": row["Image"] if isinstance(row["Image"], str) else None,
        "product_url": row["Product Url"] if "Product Url" in df.columns and isinstance(row["Product Url"], str) else None,
        "about_product": row["About Product Clean"],
        "product_specification": row["Product Specification Clean"],
        "technical_details": row["Technical Details Clean"]
    }

df_clean['metadata'] = df_clean.apply(build_metadata, axis=1)

## Embedding image and text and storing it in ChromaDB

In [51]:
# CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Setup ChromaDB persistent storage
db_path = "./amazon_product_db"
os.makedirs(db_path, exist_ok=True)

# ChromaDB Persistent Client
client = chromadb.PersistentClient(path=db_path)
collection = client.get_or_create_collection(name="amazon_products")

# Image cache folder
image_cache_dir = "./image_cache"
os.makedirs(image_cache_dir, exist_ok=True)

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 86.9MiB/s]


Prompt used in ChatGPT for the code below: "What’s the best way in Python to download images from a list of URLs and save them locally with their product IDs as filenames? Also, how do I handle failed downloads gracefully?"

In [52]:
# Function to download images
def download_image(url, unique_id):
    if not url or not isinstance(url, str) or not url.strip():
        print(f"[SKIP] Empty or invalid URL for {unique_id}")
        return None

    img_path = os.path.join(image_cache_dir, f"{unique_id}.jpg")
    if os.path.exists(img_path):
        return img_path

    try:
        headers = {"User-Agent": "Mozilla/5.0"}  # helps with 403s
        r = requests.get(url, timeout=10, headers=headers, stream=True)
        if r.status_code != 200:
            print(f"[FAIL] HTTP {r.status_code} for {unique_id} URL: {url}")
            return None

        try:
            image = Image.open(BytesIO(r.content)).convert("RGB")
        except Exception as e:
            print(f"[FAIL] PIL cannot open image for {unique_id}: {e}")
            return None

        image.save(img_path, format="JPEG", quality=90)
        return img_path

    except requests.exceptions.RequestException as e:
        print(f"[FAIL] Request error for {unique_id}: {e}")
        return None

Prompt used in ChatGPT for the code below: "I need to break long product descriptions into smaller chunks (77 tokens max) so I can embed them. Can you give me a function to chunk text into sentences or fixed-length word chunks?"

In [53]:
# Splits text into chunks that fit within CLIP's max token limit.
def chunk_text_for_clip(text, max_tokens=75):
    if not isinstance(text, str) or not text.strip():
        return []
    words = text.split()
    chunks, current_chunk = [], []
    for word in words:
        trial_chunk = current_chunk + [word]
        token_count = clip.tokenize([" ".join(trial_chunk)]).shape[1]
        if token_count <= max_tokens:
            current_chunk.append(word)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

In [54]:
# Batched Embedding Functions
def batch_text_embeddings(text_list, batch_size=128):
    embeddings = []
    for i in tqdm(range(0, len(text_list), batch_size), desc="Encoding text"):
        batch = text_list[i:i+batch_size]
        with torch.no_grad():
            tokens = clip.tokenize(batch).to(device)
            embs = model.encode_text(tokens)
            embs /= embs.norm(dim=-1, keepdim=True)
            embeddings.extend(embs.cpu().numpy())
    return embeddings

def batch_image_embeddings(image_paths, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(image_paths), batch_size), desc="Encoding images"):
        batch = image_paths[i:i+batch_size]
        processed = torch.stack([preprocess(Image.open(p)) for p in batch]).to(device)
        with torch.no_grad():
            embs = model.encode_image(processed)
            embs /= embs.norm(dim=-1, keepdim=True)
            embeddings.extend(embs.cpu().numpy())
    return embeddings

In [55]:
# Pre-processing phase
text_entries, text_ids, text_metas = [], [], []
image_entries, image_ids, image_docs, image_metas = [], [], [], []
failed_images = []

for row in tqdm(df_clean.to_dict("records"), total=len(df_clean), desc="Preparing entries"):
    uid = row["Uniq Id"]
    context_text = row["context_text"]
    metadata = row["metadata"]
    image_exists = str(row["Image Exists"]).strip().upper()
    image_url = (row["Image"] or "").strip()

    # Text chunks
    for idx, chunk in enumerate(chunk_text_for_clip(context_text)):
        text_entries.append(chunk)
        text_ids.append(f"{uid}_text_{idx}")
        text_metas.append(metadata)

    # Image download
    if image_exists == "Y" and image_url:
        img_path = download_image(image_url, uid)
        if img_path:
            image_entries.append(img_path)
            image_ids.append(f"{uid}_image")
            image_docs.append("")
            image_metas.append(metadata)
        else:
            failed_images.append({"Uniq Id": uid, "url": image_url})
            df_clean.loc[df_clean["Uniq Id"] == uid, "Image"] = "Information unavailable"

print(f"Total text chunks: {len(text_entries)}")
print(f"Total images: {len(image_entries)}")
print(f"Failed images: {len(failed_images)}")

Preparing entries:   2%|▏         | 250/10002 [00:33<3:36:03,  1.33s/it]

[FAIL] Request error for 7edbff77b5a1963eafd07e27ce66f5f3: HTTPSConnectionPool(host='images-na.ssl-images-amazon.com', port=443): Read timed out. (read timeout=10)


Preparing entries:  86%|████████▌ | 8620/10002 [15:31<25:49,  1.12s/it]

[FAIL] Request error for 8cc27788143d5a19335ff583c74b52b7: HTTPSConnectionPool(host='images-na.ssl-images-amazon.com', port=443): Read timed out. (read timeout=10)


Preparing entries: 100%|██████████| 10002/10002 [17:25<00:00,  9.57it/s]

Total text chunks: 127943
Total images: 9968
Failed images: 2


In [56]:
# Embedding Phase
all_ids, all_docs, all_metas, all_embeds = [], [], [], []

# Text embeddings
text_embs = batch_text_embeddings(text_entries, batch_size=128)
all_ids.extend(text_ids)
all_docs.extend(text_entries)
all_metas.extend(text_metas)
all_embeds.extend(text_embs)

# Image embeddings
if image_entries:
    image_embs = batch_image_embeddings(image_entries, batch_size=32)
    all_ids.extend(image_ids)
    all_docs.extend(image_docs)
    all_metas.extend(image_metas)
    all_embeds.extend(image_embs)

Encoding images: 100%|██████████| 312/312 [00:54<00:00,  5.70it/s]


In [57]:
if not (len(all_ids) == len(all_docs) == len(all_metas) == len(all_embeds)):
    raise ValueError("Length mismatch between IDs, docs, metas, and embeddings")

if not all(isinstance(e, np.ndarray) or isinstance(e, list) for e in all_embeds):
    raise ValueError("Embeddings contain non-array values")

print("Data validated — ready for ChromaDB.")

Data validated — ready for ChromaDB.


In [58]:
# Write to ChromaDB
batch_size = 500
print(f" Writing {len(all_ids)} entries to ChromaDB...")
for i in tqdm(range(0, len(all_ids), batch_size), desc="Saving to ChromaDB"):
    collection.add(
        ids=all_ids[i:i+batch_size],
        embeddings=all_embeds[i:i+batch_size],
        documents=all_docs[i:i+batch_size],
        metadatas=all_metas[i:i+batch_size]
    )

print("Finished writing to ChromaDB.")

 Writing 137911 entries to ChromaDB...


Saving to ChromaDB: 100%|██████████| 276/276 [02:44<00:00,  1.68it/s]

Finished writing to ChromaDB.


In [60]:
# make zip file to save for reuse later if needed
shutil.make_archive("/content/amazon_product_db_v1", 'zip', "/content/amazon_product_db")

'/content/amazon_product_db_v1.zip'